In [63]:
import numpy as np
import pandas as pd

In [64]:
import torch

In [65]:
import torch.nn as nn
import torch.nn.parallel
import torch.utils.data
import torch.optim as optim
from torch.autograd import Variable 

In [66]:
#movies = pd.read_csv('ml-1m/movies.dat', delimiter = '::', header = None, engine = 'python', encoding = 'latin-1')

In [67]:
#users = pd.read_csv('ml-1m/users.dat', delimiter = '::', header = None, engine = 'python', encoding = 'latin-1')

In [68]:
#ratings = pd.read_csv('ml-1m/ratings.dat', delimiter = '::', header = None, engine = 'python', encoding = 'latin-1')

In [69]:
training_df = pd.read_csv('ml-100k/u1.base', delimiter = '\t', header = None)
training_set = np.array(training_df, dtype = 'int')
test_df = pd.read_csv('ml-100k/u1.test', delimiter = '\t', header = None)
test_set = np.array(test_df, dtype = 'int')

In [70]:
training_df.head(3)

,0,1,2,3
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960


In [71]:
test_df.head(3)

,0,1,2,3
0,1,6,5,887431973
1,1,10,3,875693118
2,1,12,5,878542960


In [72]:
no_users = max(training_set[:,0].max(),test_set[:,0].max())
no_movies = max(training_set[:,1].max(),test_set[:,1].max())

In [73]:
print('No of users: ',no_users, '| No of movies: ', no_movies)

No of users:  943 | No of movies:  1682


In [74]:
def convert_data_structure(data):
    list_of_lists = []
    for i in range(1, no_users + 1):
        movies_id = data[:,1][data[:,0] == i]
        ratings = data[:,2][data[:,0] == i]
        user_row = np.zeros(no_movies)
        user_row[movies_id -1] = ratings
        list_of_lists.append(user_row)
    return list_of_lists

In [75]:
training_set = convert_data_structure(training_set)
test_set = convert_data_structure(test_set)

In [76]:
training_set

[array([5., 3., 4., ..., 0., 0., 0.]),
 array([4., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([4., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([3., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([5., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([5., 0., 0., ..., 0., 0., 0.]),
 array([0., 2., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([5., 0., 0., ..., 0., 0., 0.]),
 array([3., 0., 0., ..., 

In [77]:
#convert numpy arrays into torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [78]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

In [79]:
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

In [80]:
# Creating the class for the architecture of the restricted boltzmann neural network
class RBoltzmannArch():
    
    def __init__(self, nv, nh):
        self.W = torch.randn(nv, nh)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
    
    def sample_h(self, x):
        wx = torch.mm(x, self.W)
        activation = wx + self.a.expand_as(wx)
        prob_h_given_v = torch.sigmoid(activation)
        return prob_h_given_v, torch.bernoulli(prob_h_given_v)
    
    def sample_v(self, y):
        wy = torch.mm(y, self.W.t())
        activation = wy + self.b.expand_as(wy)
        prob_v_given_h = torch.sigmoid(activation)
        return prob_v_given_h, torch.bernoulli(prob_v_given_h)
    
    def train_rbm(self, v0, vk, ph0, phk):
        self.W += torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)

In [81]:
batch_size = 100
nv = len(training_set[0])
nh = 100 # this can be tuned later on

#initialise the object of class RBoltzmannArch
rbm = RBoltzmannArch(nv, nh)

In [82]:
#training the Restricted Boltzmann Machine
no_epochs = 10

for epoch in range(1, no_epochs+1):
    training_loss = 0
    loss_normaliser = 0.
    for batch in range(0, no_users - batch_size, batch_size):
        vk = training_set[batch:batch+batch_size]
        v0 = training_set[batch:batch+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10): # k step contrastive divergence
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train_rbm(v0, vk, ph0, phk)
        training_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        loss_normaliser += 1.
    print('epoch number: ',epoch, '|', 'training loss: ',training_loss/loss_normaliser)

epoch number:  1 | training loss:  tensor(0.3276)
epoch number:  2 | training loss:  tensor(0.2518)
epoch number:  3 | training loss:  tensor(0.2509)
epoch number:  4 | training loss:  tensor(0.2500)
epoch number:  5 | training loss:  tensor(0.2482)
epoch number:  6 | training loss:  tensor(0.2467)
epoch number:  7 | training loss:  tensor(0.2503)
epoch number:  8 | training loss:  tensor(0.2465)
epoch number:  9 | training loss:  tensor(0.2473)
epoch number:  10 | training loss:  tensor(0.2507)


In [83]:
# testing the restricted boltzmann algorithm against the test set
test_loss = 0
loss_normaliser = 0.
for user_id in range(no_users):
    v = training_set[user_id:user_id+1]
    vt = test_set[user_id:user_id+1]
    if len(vt[vt>=0]) > 0:
        _, h = rbm.sample_h(v)
        _, v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        loss_normaliser += 1.
print('test loss: ',test_loss/loss_normaliser)

test loss:  tensor(0.2376)


In [84]:
# this shows that the recommender system is correctly predicting 1 or 0 (like or dislike) with an accuracy of 76%